# Lab 6.2: Counter-Current Exchange Simulator
## Chapter 6: The Oxygen Revolution - Respiratory Systems

### 🎯 Learning Objectives
By completing this interactive lab, you will:
- Understand the principles of counter-current vs concurrent flow
- Calculate oxygen extraction efficiency for different flow patterns
- Visualize concentration gradients along exchange surfaces
- Appreciate why counter-current exchange is evolutionarily superior
- Apply these principles to both gills and heat exchangers

### 📖 Connection to Chapter 6
This lab explores concepts from **Section 6.2: Gills - Mastering Underwater Respiration**, where we learned that counter-current flow achieves 80-85% oxygen extraction efficiency compared to only 25% with concurrent flow.

**Why does this matter?**
- Water contains ~30× less oxygen than air
- Fish must extract maximum oxygen from every drop
- Counter-current design maintains concentration gradient across entire exchange surface
- This principle appears throughout biology: gills, kidneys, heat exchangers

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from ipywidgets import interact, interactive, FloatSlider, Dropdown, VBox, HBox, Button, Output, HTML, IntSlider
import io
import base64
from datetime import datetime
from IPython.display import display, Image as IPImage

print("✓ All libraries loaded successfully!")
print("Ready to explore counter-current exchange...")

## Part 1: Understanding Exchange Patterns

### The Fundamental Difference

Imagine oxygen transferring between two fluids flowing past each other. The pattern of flow determines efficiency:

**Concurrent Flow** (Parallel, same direction):
```
Water → → → → →
Blood → → → → →
```
- Both fluids flow in the same direction
- Gradient decreases along exchanger
- Exchange stops when equilibrium reached
- Maximum efficiency: ~25%

**Counter-Current Flow** (Opposite directions):
```
Water → → → → →
Blood ← ← ← ← ←
```
- Fluids flow in opposite directions
- Gradient maintained along entire exchanger
- Can approach 100% equilibration
- Real efficiency: 80-85% in fish gills

Let's see why this difference is so dramatic!

In [ ]:
# Core exchange simulation functions
def simulate_exchange(flow_type='counter-current', exchanger_length=10, 
                     water_o2_in=60, blood_o2_in=20, 
                     water_flow=1.0, blood_flow=1.0,
                     exchange_coefficient=0.5):
    """
    Simulate oxygen exchange in gill-like structure
    
    Parameters:
    -----------
    flow_type : str
        'counter-current' or 'concurrent'
    exchanger_length : int
        Number of segments in exchanger
    water_o2_in : float
        Incoming water oxygen (mmHg)
    blood_o2_in : float
        Incoming blood oxygen (mmHg)
    water_flow : float
        Relative water flow rate
    blood_flow : float
        Relative blood flow rate
    exchange_coefficient : float
        Rate of exchange between fluids
    
    Returns:
    --------
    dict : Contains position arrays and O2 concentrations
    """
    n = exchanger_length
    positions = np.linspace(0, 1, n)
    
    # Initialize arrays
    water_o2 = np.zeros(n)
    blood_o2 = np.zeros(n)
    
    if flow_type == 'concurrent':
        # Both flow left to right
        water_o2[0] = water_o2_in
        blood_o2[0] = blood_o2_in
        
        for i in range(1, n):
            # Calculate exchange based on concentration difference
            gradient = water_o2[i-1] - blood_o2[i-1]
            exchange = exchange_coefficient * gradient
            
            # Update concentrations
            water_o2[i] = water_o2[i-1] - exchange / water_flow
            blood_o2[i] = blood_o2[i-1] + exchange / blood_flow
            
            # Prevent impossible values
            if water_o2[i] < blood_o2[i]:
                equilibrium = (water_o2[i] + blood_o2[i]) / 2
                water_o2[i] = equilibrium
                blood_o2[i] = equilibrium
    
    else:  # counter-current
        # Water flows left to right, blood flows right to left
        # Iterate to find steady-state solution
        water_o2[0] = water_o2_in
        blood_o2[-1] = blood_o2_in
        
        # Iterative solution (simplified)
        for iteration in range(100):  # Convergence iterations
            old_water = water_o2.copy()
            old_blood = blood_o2.copy()
            
            # Water flow (left to right)
            for i in range(1, n):
                gradient = water_o2[i-1] - blood_o2[i]
                exchange = exchange_coefficient * gradient
                water_o2[i] = water_o2[i-1] - exchange / water_flow
            
            # Blood flow (right to left)
            for i in range(n-2, -1, -1):
                gradient = water_o2[i] - blood_o2[i+1]
                exchange = exchange_coefficient * gradient
                blood_o2[i] = blood_o2[i+1] + exchange / blood_flow
            
            # Check convergence
            if np.max(np.abs(water_o2 - old_water)) < 0.01:
                break
    
    # Calculate efficiency
    oxygen_extracted = water_o2[0] - water_o2[-1]
    maximum_possible = water_o2[0] - blood_o2_in
    efficiency = (oxygen_extracted / maximum_possible) * 100 if maximum_possible > 0 else 0
    
    return {
        'positions': positions,
        'water_o2': water_o2,
        'blood_o2': blood_o2,
        'efficiency': efficiency,
        'oxygen_extracted': oxygen_extracted,
        'final_blood_o2': blood_o2[0] if flow_type == 'counter-current' else blood_o2[-1]
    }

# Test the simulator
test_result = simulate_exchange()
print(f"Test simulation complete:")
print(f"  Efficiency: {test_result['efficiency']:.1f}%")
print(f"  Oxygen extracted: {test_result['oxygen_extracted']:.1f} mmHg")
print("\n✓ Exchange simulator ready!")

## Part 2: Interactive Exchange Simulator

Explore how different parameters affect exchange efficiency:

In [ ]:
# Interactive visualization class
class ExchangeVisualizer:
    def __init__(self):
        self.comparison_data = []
        
    def visualize_exchange(self, flow_type, water_o2_in, blood_o2_in, 
                          water_flow, blood_flow, exchange_rate):
        """
        Create interactive visualization of exchange process
        """
        # Run simulation
        result = simulate_exchange(
            flow_type=flow_type,
            exchanger_length=50,
            water_o2_in=water_o2_in,
            blood_o2_in=blood_o2_in,
            water_flow=water_flow,
            blood_flow=blood_flow,
            exchange_coefficient=exchange_rate
        )
        
        # Create visualization
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Oxygen Concentration Along Exchanger',
                'Concentration Gradient',
                'Exchange Efficiency',
                'Flow Pattern Diagram'
            ),
            specs=[
                [{'type': 'scatter'}, {'type': 'scatter'}],
                [{'type': 'indicator'}, {'type': 'scatter'}]
            ],
            vertical_spacing=0.15,
            horizontal_spacing=0.12
        )
        
        # 1. Concentration profiles
        fig.add_trace(
            go.Scatter(
                x=result['positions'],
                y=result['water_o2'],
                mode='lines+markers',
                name='Water O₂',
                line=dict(color='#0077B6', width=3),
                marker=dict(size=6)
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=result['positions'],
                y=result['blood_o2'],
                mode='lines+markers',
                name='Blood O₂',
                line=dict(color='#C1121F', width=3),
                marker=dict(size=6)
            ),
            row=1, col=1
        )
        
        # 2. Concentration gradient
        gradient = result['water_o2'] - result['blood_o2']
        fig.add_trace(
            go.Scatter(
                x=result['positions'],
                y=gradient,
                fill='tozeroy',
                name='Gradient',
                line=dict(color='#2A9D8F', width=3),
                fillcolor='rgba(42, 157, 143, 0.3)'
            ),
            row=1, col=2
        )
        
        # Add zero line
        fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=2)
        
        # 3. Efficiency indicator
        color = '#2A9D8F' if result['efficiency'] > 60 else '#F4A261' if result['efficiency'] > 30 else '#E76F51'
        
        fig.add_trace(
            go.Indicator(
                mode="gauge+number+delta",
                value=result['efficiency'],
                title={'text': "Extraction Efficiency"},
                number={'suffix': "%"},
                delta={'reference': 85 if flow_type == 'counter-current' else 25},
                gauge={
                    'axis': {'range': [None, 100]},
                    'bar': {'color': color},
                    'steps': [
                        {'range': [0, 30], 'color': "#FFE5E5"},
                        {'range': [30, 60], 'color': "#FFE1CC"},
                        {'range': [60, 100], 'color': "#D4F1E8"}
                    ],
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': 85
                    }
                }
            ),
            row=2, col=1
        )
        
        # 4. Flow pattern diagram
        if flow_type == 'concurrent':
            # Concurrent flow arrows
            arrow_positions = [0.2, 0.4, 0.6, 0.8]
            fig.add_trace(
                go.Scatter(
                    x=arrow_positions,
                    y=[1.5]*4,
                    mode='markers+text',
                    marker=dict(size=20, color='#0077B6', symbol='arrow-right'),
                    text=['→']*4,
                    textposition='middle center',
                    textfont=dict(size=20),
                    name='Water flow',
                    showlegend=False
                ),
                row=2, col=2
            )
            
            fig.add_trace(
                go.Scatter(
                    x=arrow_positions,
                    y=[0.5]*4,
                    mode='markers+text',
                    marker=dict(size=20, color='#C1121F', symbol='arrow-right'),
                    text=['→']*4,
                    textposition='middle center',
                    textfont=dict(size=20),
                    name='Blood flow',
                    showlegend=False
                ),
                row=2, col=2
            )
        else:
            # Counter-current flow arrows
            arrow_positions = [0.2, 0.4, 0.6, 0.8]
            fig.add_trace(
                go.Scatter(
                    x=arrow_positions,
                    y=[1.5]*4,
                    mode='markers+text',
                    marker=dict(size=20, color='#0077B6', symbol='arrow-right'),
                    text=['→']*4,
                    textposition='middle center',
                    textfont=dict(size=20),
                    name='Water flow',
                    showlegend=False
                ),
                row=2, col=2
            )
            
            fig.add_trace(
                go.Scatter(
                    x=arrow_positions,
                    y=[0.5]*4,
                    mode='markers+text',
                    marker=dict(size=20, color='#C1121F', symbol='arrow-left'),
                    text=['←']*4,
                    textposition='middle center',
                    textfont=dict(size=20),
                    name='Blood flow',
                    showlegend=False
                ),
                row=2, col=2
            )
        
        # Add labels
        fig.add_annotation(
            text="Water", x=0.1, y=1.5, showarrow=False,
            font=dict(size=14, color='#0077B6', family='Arial Black'),
            row=2, col=2
        )
        fig.add_annotation(
            text="Blood", x=0.1, y=0.5, showarrow=False,
            font=dict(size=14, color='#C1121F', family='Arial Black'),
            row=2, col=2
        )
        
        # Update layout
        fig.update_xaxes(title_text="Position Along Exchanger", row=1, col=1)
        fig.update_xaxes(title_text="Position Along Exchanger", row=1, col=2)
        fig.update_xaxes(showticklabels=False, row=2, col=2, range=[0, 1])
        
        fig.update_yaxes(title_text="O₂ Concentration (mmHg)", row=1, col=1)
        fig.update_yaxes(title_text="Gradient (mmHg)", row=1, col=2)
        fig.update_yaxes(showticklabels=False, row=2, col=2, range=[0, 2])
        
        fig.update_layout(
            height=700,
            title_text=f"<b>{flow_type.title()} Exchange Analysis</b>",
            title_x=0.5,
            showlegend=True,
            legend=dict(x=0.02, y=0.98)
        )
        
        # Print summary
        print("\n" + "="*70)
        print(f"EXCHANGE ANALYSIS: {flow_type.upper()}")
        print("="*70)
        print(f"\nExtraction Efficiency: {result['efficiency']:.1f}%")
        print(f"Oxygen Extracted: {result['oxygen_extracted']:.1f} mmHg")
        print(f"Final Blood O₂: {result['final_blood_o2']:.1f} mmHg")
        print(f"\nInput Conditions:")
        print(f"  Water O₂ in: {water_o2_in} mmHg")
        print(f"  Blood O₂ in: {blood_o2_in} mmHg")
        print(f"  Water flow rate: {water_flow:.1f}x")
        print(f"  Blood flow rate: {blood_flow:.1f}x")
        
        if flow_type == 'counter-current':
            print("\n✓ Counter-current maintains gradient along entire exchanger!")
            if result['efficiency'] > 70:
                print("✓ Achieving shark-like efficiency (60-70%)!")
        else:
            print("\n⚠ Concurrent flow: gradient collapses to zero!")
            print("  Exchange stops when concentrations equilibrate.")
        
        print("\n" + "="*70)
        
        fig.show()
        
        # Store for comparison
        self.current_result = {
            'flow_type': flow_type,
            'efficiency': result['efficiency'],
            'oxygen_extracted': result['oxygen_extracted'],
            'water_o2_in': water_o2_in,
            'blood_o2_in': blood_o2_in
        }
    
    def add_to_comparison(self):
        """Add current result to comparison"""
        if hasattr(self, 'current_result'):
            self.comparison_data.append(self.current_result.copy())
            print(f"✓ Added to comparison (Total: {len(self.comparison_data)})")
    
    def show_comparison(self):
        """Display comparison of different configurations"""
        if len(self.comparison_data) < 2:
            print("⚠ Add at least 2 configurations to compare!")
            return
        
        df = pd.DataFrame(self.comparison_data)
        
        fig = px.bar(
            df, 
            x=df.index,
            y='efficiency',
            color='flow_type',
            title='<b>Exchange Efficiency Comparison</b>',
            labels={'efficiency': 'Extraction Efficiency (%)', 'index': 'Configuration'},
            text='efficiency'
        )
        
        fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
        fig.update_layout(height=500)
        fig.show()
        
        display(df)
    
    def clear_comparison(self):
        """Clear comparison data"""
        self.comparison_data = []
        print("✓ Comparison cleared")

# Create visualizer instance
visualizer = ExchangeVisualizer()

print("\n✓ Exchange visualizer ready!")
print("Run the next cell to start exploring...")

In [ ]:
# Interactive widget interface
flow_dropdown = Dropdown(
    options=['counter-current', 'concurrent'],
    value='counter-current',
    description='Flow Pattern:',
    style={'description_width': '120px'}
)

water_o2_slider = FloatSlider(
    value=60,
    min=20,
    max=100,
    step=5,
    description='Water O₂ (mmHg):',
    style={'description_width': '150px'}
)

blood_o2_slider = FloatSlider(
    value=20,
    min=0,
    max=60,
    step=5,
    description='Blood O₂ (mmHg):',
    style={'description_width': '150px'}
)

water_flow_slider = FloatSlider(
    value=1.0,
    min=0.5,
    max=3.0,
    step=0.1,
    description='Water Flow Rate:',
    style={'description_width': '150px'},
    readout_format='.1f'
)

blood_flow_slider = FloatSlider(
    value=1.0,
    min=0.5,
    max=3.0,
    step=0.1,
    description='Blood Flow Rate:',
    style={'description_width': '150px'},
    readout_format='.1f'
)

exchange_rate_slider = FloatSlider(
    value=0.5,
    min=0.1,
    max=1.0,
    step=0.1,
    description='Exchange Rate:',
    style={'description_width': '150px'},
    readout_format='.1f'
)

# Create interactive output
output = interactive(
    visualizer.visualize_exchange,
    flow_type=flow_dropdown,
    water_o2_in=water_o2_slider,
    blood_o2_in=blood_o2_slider,
    water_flow=water_flow_slider,
    blood_flow=blood_flow_slider,
    exchange_rate=exchange_rate_slider
)

# Add comparison buttons
add_button = Button(description='Add to Comparison', button_style='success', icon='plus')
add_button.on_click(lambda b: visualizer.add_to_comparison())

compare_button = Button(description='Show Comparison', button_style='info', icon='bar-chart')
compare_button.on_click(lambda b: visualizer.show_comparison())

clear_button = Button(description='Clear Comparison', button_style='warning', icon='trash')
clear_button.on_click(lambda b: visualizer.clear_comparison())

button_box = HBox([add_button, compare_button, clear_button])

# Display interface
display(HTML("<h3>🔬 Interactive Counter-Current Exchange Simulator</h3>"))
display(HTML("<p><i>Explore how flow pattern affects oxygen extraction efficiency</i></p>"))
display(output)
display(HTML("<br><h4>Comparison Tools:</h4>"))
display(button_box)

## Part 3: Real Species Comparison

Let's compare actual fish respiratory systems using data from Chapter 6:

In [ ]:
# Species database from Chapter 6
fish_species = {
    'Great White Shark': {
        'gill_area': 7.0,  # m²
        'extraction_efficiency': 65,  # %
        'water_o2': 45,  # mmHg (seawater)
        'blood_o2_in': 20,  # mmHg
        'ventilation': 'Ram ventilation (continuous swimming)',
        'lifestyle': 'Active predator',
        'description': 'High-performance counter-current gills. Must swim continuously '
                      'to maintain water flow. Achieves 60-70% extraction efficiency.'
    },
    'Bottom-Dwelling Skate': {
        'gill_area': 2.5,  # m²
        'extraction_efficiency': 55,  # %
        'water_o2': 45,  # mmHg
        'blood_o2_in': 25,  # mmHg
        'ventilation': 'Buccal pumping (active)',
        'lifestyle': 'Benthic (bottom-dwelling)',
        'description': 'Modified gill ventilation for bottom-dwelling. Spiracles '
                      'allow water intake while resting on substrate.'
    },
    'Tuna': {
        'gill_area': 8.5,  # m²
        'extraction_efficiency': 80,  # %
        'water_o2': 45,  # mmHg
        'blood_o2_in': 15,  # mmHg
        'ventilation': 'Obligate ram ventilation',
        'lifestyle': 'Extreme performance predator',
        'description': 'Ultimate fish respiratory performance. Enormous gill area, '
                      'extremely thin membranes, high blood flow. Cannot stop swimming.'
    },
    'Lungfish': {
        'gill_area': 1.2,  # m² (reduced)
        'extraction_efficiency': 35,  # % (gills alone)
        'water_o2': 30,  # mmHg (hypoxic water)
        'blood_o2_in': 20,  # mmHg
        'ventilation': 'Dual system: gills + primitive lungs',
        'lifestyle': 'Hypoxia-tolerant, can breathe air',
        'description': 'Transitional form with both gills and lungs. Switches between '
                      'aquatic and aerial respiration based on water oxygen levels.'
    },
    'Goldfish (Hypoxia)': {
        'gill_area': 0.8,  # m²
        'extraction_efficiency': 70,  # % (enhanced in hypoxia)
        'water_o2': 20,  # mmHg (low oxygen)
        'blood_o2_in': 15,  # mmHg
        'ventilation': 'Buccal pumping (increased rate)',
        'lifestyle': 'Hypoxia-tolerant',
        'description': 'Can survive low oxygen by increasing gill area (more lamellae '
                      'recruited), higher ventilation rate, metabolic suppression.'
    }
}

# Create comparison visualization
def compare_fish_species():
    """
    Compare respiratory performance across fish species
    """
    species_names = list(fish_species.keys())
    gill_areas = [fish_species[s]['gill_area'] for s in species_names]
    efficiencies = [fish_species[s]['extraction_efficiency'] for s in species_names]
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Gill Surface Area', 'Extraction Efficiency'),
        specs=[[{'type': 'bar'}, {'type': 'bar'}]]
    )
    
    # Gill area comparison
    fig.add_trace(
        go.Bar(
            x=species_names,
            y=gill_areas,
            marker_color='#0077B6',
            text=[f"{a:.1f} m²" for a in gill_areas],
            textposition='auto',
            name='Gill Area'
        ),
        row=1, col=1
    )
    
    # Efficiency comparison
    colors = ['#2A9D8F' if e > 60 else '#F4A261' if e > 40 else '#E76F51' 
              for e in efficiencies]
    
    fig.add_trace(
        go.Bar(
            x=species_names,
            y=efficiencies,
            marker_color=colors,
            text=[f"{e}%" for e in efficiencies],
            textposition='auto',
            name='Efficiency'
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        height=500,
        title_text='<b>Fish Respiratory System Comparison (Chapter 6 Data)</b>',
        showlegend=False
    )
    
    fig.update_xaxes(tickangle=45)
    fig.update_yaxes(title_text="Surface Area (m²)", row=1, col=1)
    fig.update_yaxes(title_text="Efficiency (%)", row=1, col=2)
    
    fig.show()
    
    # Print detailed comparison
    print("\n" + "="*70)
    print("FISH RESPIRATORY SYSTEMS - DETAILED COMPARISON")
    print("="*70)
    
    for species, data in fish_species.items():
        print(f"\n{species}:")
        print(f"  Lifestyle: {data['lifestyle']}")
        print(f"  Gill Area: {data['gill_area']:.1f} m²")
        print(f"  Extraction Efficiency: {data['extraction_efficiency']}%")
        print(f"  Ventilation: {data['ventilation']}")
        print(f"  {data['description']}")
    
    print("\n" + "="*70)
    print("KEY INSIGHTS:")
    print("="*70)
    print("✓ Active predators (sharks, tuna) have largest gill areas")
    print("✓ Tuna achieves 80% efficiency - nearly theoretical maximum!")
    print("✓ Hypoxia-tolerant species can boost efficiency when needed")
    print("✓ Lungfish trades gill efficiency for air-breathing capability")
    print("✓ All use counter-current exchange for maximum extraction")

# Run comparison
compare_fish_species()

## Part 4: Challenge Problems

Test your understanding with these challenges:

### Challenge 1: Design for Hypoxic Water

**Scenario**: A fish lives in a pond where oxygen levels drop to 20 mmHg (vs 45 mmHg in normal seawater).

**Questions**:
1. How does reduced water oxygen affect extraction efficiency?
2. What adaptations would help maintain oxygen delivery?
3. Compare counter-current vs concurrent flow in hypoxic conditions
4. Why is counter-current exchange even more important in low oxygen?

<details>
<summary>Click for hints</summary>

- Use the simulator with water_o2_in=20
- Try increasing ventilation rate (water_flow)
- Compare efficiencies with both flow patterns
- Consider the goldfish hypoxia adaptations above
</details>

In [ ]:
# Challenge 1 workspace
print("CHALLENGE 1: SURVIVAL IN HYPOXIC WATER")
print("="*70)

# Normal conditions
normal_result = simulate_exchange(
    flow_type='counter-current',
    water_o2_in=45,
    blood_o2_in=20,
    water_flow=1.0
)

# Hypoxic conditions - basic response
hypoxic_basic = simulate_exchange(
    flow_type='counter-current',
    water_o2_in=20,
    blood_o2_in=20,
    water_flow=1.0
)

# Hypoxic conditions - with increased ventilation
hypoxic_adapted = simulate_exchange(
    flow_type='counter-current',
    water_o2_in=20,
    blood_o2_in=15,  # Lower due to metabolic suppression
    water_flow=2.5  # Increased ventilation
)

# Compare with concurrent flow in hypoxia
hypoxic_concurrent = simulate_exchange(
    flow_type='concurrent',
    water_o2_in=20,
    blood_o2_in=15,
    water_flow=2.5
)

print("\nNORMAL CONDITIONS (45 mmHg O₂):")
print(f"  Efficiency: {normal_result['efficiency']:.1f}%")
print(f"  Blood O₂ achieved: {normal_result['final_blood_o2']:.1f} mmHg")

print("\nHYPOXIC - NO ADAPTATION (20 mmHg O₂):")
print(f"  Efficiency: {hypoxic_basic['efficiency']:.1f}%")
print(f"  Blood O₂ achieved: {hypoxic_basic['final_blood_o2']:.1f} mmHg")
print("  ⚠ Gradient too small - fish would struggle!")

print("\nHYPOXIC - WITH ADAPTATIONS:")
print(f"  Efficiency: {hypoxic_adapted['efficiency']:.1f}%")
print(f"  Blood O₂ achieved: {hypoxic_adapted['final_blood_o2']:.1f} mmHg")
print("  ✓ Increased ventilation + metabolic suppression helps!")

print("\nHYPOXIC - CONCURRENT FLOW (for comparison):")
print(f"  Efficiency: {hypoxic_concurrent['efficiency']:.1f}%")
print(f"  Blood O₂ achieved: {hypoxic_concurrent['final_blood_o2']:.1f} mmHg")
print("  ⚠ Much worse! Counter-current is ESSENTIAL in hypoxia!")

print("\n" + "="*70)
print("WHY COUNTER-CURRENT IS CRITICAL IN LOW OXYGEN:")
print("="*70)
print("• Smaller gradient requires maximum extraction efficiency")
print("• Counter-current extracts", f"{hypoxic_adapted['efficiency']:.0f}%",
      "vs", f"{hypoxic_concurrent['efficiency']:.0f}%", "for concurrent")
print("• This difference means survival vs death!")
print("• Real fish adaptations: increase gill area, boost ventilation, suppress metabolism")

# Your analysis:
print("\nYOUR ANALYSIS:")
print("What other adaptations might help fish survive hypoxia?")
# Add your thoughts

### Challenge 2: Flow Rate Optimization

**Scenario**: A fish can vary its ventilation rate (how fast water flows over gills).

**Questions**:
1. How does water flow rate affect extraction efficiency?
2. Is there an optimal flow rate?
3. What happens if blood flow doesn't match water flow?
4. Why do active fish breathe faster?

<details>
<summary>Click for hints</summary>

- Test multiple water_flow values from 0.5 to 3.0
- Try matching vs mismatching blood_flow
- Consider energy cost of pumping water
- Higher flow = more oxygen delivered but more energy cost
</details>

In [ ]:
# Challenge 2 workspace
print("CHALLENGE 2: FLOW RATE OPTIMIZATION")
print("="*70)

# Test range of flow rates
flow_rates = np.linspace(0.5, 3.0, 10)
efficiencies = []
oxygen_delivered = []

for flow in flow_rates:
    result = simulate_exchange(
        flow_type='counter-current',
        water_o2_in=45,
        blood_o2_in=20,
        water_flow=flow,
        blood_flow=flow  # Matched flows
    )
    efficiencies.append(result['efficiency'])
    # Total O2 delivered = extraction × flow rate
    oxygen_delivered.append(result['oxygen_extracted'] * flow)

# Create optimization plot
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Extraction Efficiency vs Flow Rate', 
                   'Total Oxygen Delivered vs Flow Rate')
)

fig.add_trace(
    go.Scatter(
        x=flow_rates,
        y=efficiencies,
        mode='lines+markers',
        line=dict(color='#2A9D8F', width=3),
        marker=dict(size=8),
        name='Efficiency'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=flow_rates,
        y=oxygen_delivered,
        mode='lines+markers',
        line=dict(color='#E76F51', width=3),
        marker=dict(size=8),
        name='O₂ Delivered'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Water Flow Rate (relative)", row=1, col=1)
fig.update_xaxes(title_text="Water Flow Rate (relative)", row=1, col=2)
fig.update_yaxes(title_text="Efficiency (%)", row=1, col=1)
fig.update_yaxes(title_text="O₂ Delivered (arbitrary units)", row=1, col=2)

fig.update_layout(
    height=400,
    title_text='<b>Flow Rate Optimization Analysis</b>',
    showlegend=False
)

fig.show()

print("\nKEY FINDINGS:")
print("="*70)
print(f"• Efficiency DECREASES with higher flow rate")
print(f"  (from {efficiencies[0]:.1f}% at low flow to {efficiencies[-1]:.1f}% at high flow)")
print(f"\n• But TOTAL oxygen delivered INCREASES!")
print(f"  (efficiency × flow rate = actual O₂ uptake)")
print(f"\n• Trade-off: Lower efficiency but more total oxygen")
print(f"  Active fish accept lower % extraction for higher total delivery")

# Test mismatched flows
print("\n" + "="*70)
print("EFFECT OF MISMATCHED FLOWS:")
print("="*70)

matched = simulate_exchange(water_flow=2.0, blood_flow=2.0)
high_water = simulate_exchange(water_flow=3.0, blood_flow=1.0)
high_blood = simulate_exchange(water_flow=1.0, blood_flow=3.0)

print(f"\nMatched flows (2.0 / 2.0): {matched['efficiency']:.1f}% efficiency")
print(f"High water flow (3.0 / 1.0): {high_water['efficiency']:.1f}% efficiency")
print(f"High blood flow (1.0 / 3.0): {high_blood['efficiency']:.1f}% efficiency")

print("\n✓ BEST performance with matched flow rates!")
print("  This is why fish coordinate ventilation with cardiac output")

# Your analysis:
print("\nYOUR ANALYSIS:")
print("Why do you think active fish (like sharks and tuna) breathe faster?")
# Add your reasoning

### Challenge 3: Biomimetic Heat Exchanger

**Scenario**: Design a heat exchanger for a building's ventilation system using counter-current principles.

**Requirements**:
- Recover heat from exhaust air to warm incoming fresh air
- Exhaust air: 22°C
- Incoming air: 5°C
- Goal: Raise incoming air to at least 18°C

**Questions**:
1. Why is counter-current flow essential?
2. What extraction efficiency would you need?
3. How does this relate to fish gill design?
4. What engineering challenges exist?

<details>
<summary>Click for hints</summary>

- Heat transfer follows same principles as oxygen exchange
- Temperature is analogous to oxygen concentration
- Gradient = 22°C - 5°C = 17°C
- Need to increase incoming air by 13°C (18°C - 5°C)
- Calculate: 13/17 = 76% efficiency needed
</details>

In [ ]:
# Challenge 3 workspace
print("CHALLENGE 3: BIOMIMETIC HEAT EXCHANGER DESIGN")
print("="*70)

# Heat exchanger parameters (analogous to gill)
exhaust_temp = 22  # °C (analogous to water O2)
incoming_temp = 5   # °C (analogous to blood O2)
target_temp = 18    # °C (goal)

# Calculate required efficiency
max_gradient = exhaust_temp - incoming_temp
required_increase = target_temp - incoming_temp
required_efficiency = (required_increase / max_gradient) * 100

print("DESIGN REQUIREMENTS:")
print(f"  Exhaust air temperature: {exhaust_temp}°C")
print(f"  Incoming air temperature: {incoming_temp}°C")
print(f"  Target incoming temperature: {target_temp}°C")
print(f"  Maximum gradient: {max_gradient}°C")
print(f"  Required temperature increase: {required_increase}°C")
print(f"\n  REQUIRED EFFICIENCY: {required_efficiency:.1f}%")

# Simulate both flow patterns
print("\n" + "="*70)
print("COMPARING FLOW PATTERNS:")
print("="*70)

# Counter-current heat exchanger
counter = simulate_exchange(
    flow_type='counter-current',
    water_o2_in=exhaust_temp,
    blood_o2_in=incoming_temp,
    water_flow=1.0,
    blood_flow=1.0,
    exchange_coefficient=0.8  # High exchange for heat
)

# Concurrent heat exchanger
concurrent = simulate_exchange(
    flow_type='concurrent',
    water_o2_in=exhaust_temp,
    blood_o2_in=incoming_temp,
    water_flow=1.0,
    blood_flow=1.0,
    exchange_coefficient=0.8
)

counter_final_temp = incoming_temp + (counter['oxygen_extracted'])
concurrent_final_temp = incoming_temp + (concurrent['oxygen_extracted'])

print(f"\nCOUNTER-CURRENT DESIGN:")
print(f"  Efficiency: {counter['efficiency']:.1f}%")
print(f"  Final incoming air temp: {counter_final_temp:.1f}°C")
if counter_final_temp >= target_temp:
    print(f"  ✓ MEETS TARGET! ({target_temp}°C)")
else:
    print(f"  ⚠ Below target ({target_temp}°C)")

print(f"\nCONCURRENT DESIGN:")
print(f"  Efficiency: {concurrent['efficiency']:.1f}%")
print(f"  Final incoming air temp: {concurrent_final_temp:.1f}°C")
if concurrent_final_temp >= target_temp:
    print(f"  ✓ MEETS TARGET! ({target_temp}°C)")
else:
    print(f"  ⚠ FAILS - Below target ({target_temp}°C)")

# Energy savings calculation
print("\n" + "="*70)
print("ENERGY SAVINGS ANALYSIS:")
print("="*70)

counter_savings = (counter_final_temp - incoming_temp) / max_gradient * 100
concurrent_savings = (concurrent_final_temp - incoming_temp) / max_gradient * 100

print(f"\nCounter-current: {counter_savings:.1f}% heating cost reduction")
print(f"Concurrent: {concurrent_savings:.1f}% heating cost reduction")
print(f"\nAdvantage: {counter_savings - concurrent_savings:.1f}% more energy saved!")

print("\n" + "="*70)
print("LESSONS FROM FISH GILLS:")
print("="*70)
print("✓ Counter-current design is ESSENTIAL for high efficiency")
print("✓ Same principle works for heat, oxygen, or any gradient")
print("✓ Fish evolved this 500 million years ago!")
print("✓ Modern buildings use gill-inspired heat recovery ventilators (HRVs)")
print("✓ Can achieve 80-95% heat recovery with counter-current design")

print("\n" + "="*70)
print("ENGINEERING CHALLENGES:")
print("="*70)
print("• Thin membranes for good heat transfer (like gill lamellae)")
print("• Large surface area in compact space (like folded gills)")
print("• Prevent air mixing between streams (like gill architecture)")
print("• Moisture management (gills handle this naturally)")
print("• Fouling prevention (gills have mucus & immune defenses)")

# Your biomimetic design:
print("\nYOUR BIOMIMETIC IMPROVEMENTS:")
print("What other gill features could improve heat exchanger design?")
# Add your ideas

## Part 5: Export Your Work

In [ ]:
# Export system
import os

def export_lab_results():
    """
    Export comparison data and visualizations
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    export_dir = "/content"
    os.makedirs(export_dir, exist_ok=True)
    
    # Export comparison data
    if visualizer.comparison_data:
        df = pd.DataFrame(visualizer.comparison_data)
        csv_file = f"{export_dir}/lab_6_2_comparison_{timestamp}.csv"
        df.to_csv(csv_file, index=False)
        print(f"✓ Comparison data saved: {csv_file}")
    
    # Create summary figure
    fig = compare_flow_patterns_summary()
    
    html_file = f"{export_dir}/lab_6_2_summary_{timestamp}.html"
    fig.write_html(html_file)
    print(f"✓ Summary saved: {html_file}")
    
    try:
        png_file = f"{export_dir}/lab_6_2_summary_{timestamp}.png"
        fig.write_image(png_file, width=1200, height=800)
        print(f"✓ PNG saved: {png_file}")
    except:
        print("⚠ PNG export requires kaleido: pip install kaleido")
    
    print("\n✓ Export complete! Files in /content")

def compare_flow_patterns_summary():
    """Create summary comparison figure"""
    # Compare counter-current vs concurrent at different gradients
    gradients = [20, 40, 60, 80]
    counter_effs = []
    concurrent_effs = []
    
    for grad in gradients:
        counter = simulate_exchange(flow_type='counter-current', 
                                   water_o2_in=grad+20, blood_o2_in=20)
        concurrent = simulate_exchange(flow_type='concurrent',
                                      water_o2_in=grad+20, blood_o2_in=20)
        counter_effs.append(counter['efficiency'])
        concurrent_effs.append(concurrent['efficiency'])
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=gradients, y=counter_effs,
        mode='lines+markers',
        name='Counter-Current',
        line=dict(color='#2A9D8F', width=4),
        marker=dict(size=12)
    ))
    
    fig.add_trace(go.Scatter(
        x=gradients, y=concurrent_effs,
        mode='lines+markers',
        name='Concurrent',
        line=dict(color='#E76F51', width=4),
        marker=dict(size=12)
    ))
    
    fig.update_layout(
        title='<b>Lab 6.2 Summary: Counter-Current Superiority</b>',
        xaxis_title='Concentration Gradient (mmHg)',
        yaxis_title='Extraction Efficiency (%)',
        height=600
    )
    
    return fig

# Create export button
export_button = Button(
    description='📥 Export Results',
    button_style='success',
    icon='download',
    layout={'width': '200px', 'height': '40px'}
)
export_button.on_click(lambda b: export_lab_results())

display(HTML("<h3>Export Your Work</h3>"))
display(export_button)

## Summary & Reflection

### What You've Learned

1. ✅ Counter-current exchange achieves 80-85% efficiency vs 25% for concurrent
2. ✅ Gradient maintenance along entire exchanger is the key advantage
3. ✅ Flow rate optimization involves trade-offs between efficiency and total delivery
4. ✅ This principle applies to heat exchangers, kidneys, and other biological systems
5. ✅ Fish evolved this optimal solution 500 million years ago!

### Connection to Chapter 6

This lab reinforced concepts from **Section 6.2: Gills - Mastering Underwater Respiration**:
- Why counter-current flow is evolutionarily superior
- How sharks achieve 60-70% extraction efficiency
- Why active fish need large gill areas and high flow rates
- Applications to biomimetic technology

### Real-World Applications

Counter-current exchange principles are used in:
- **Building ventilation** (heat recovery ventilators)
- **Industrial processes** (chemical reactors)
- **Refrigeration** (efficient heat transfer)
- **Kidney dialysis** (waste removal)
- **Bird legs** (heat conservation in cold water)

### Next Steps

Continue to **Lab 6.3: Respiratory Performance Across Environments** to explore how vertebrates adapt to extreme conditions!

---

*This interactive lab is part of "A Pattern Hunter's Guide to Comparative Anatomy"*  
*Chapter 6: The Oxygen Revolution - Respiratory Systems*